# Gyakorló feladat
Az https://archive.ics.uci.edu/ml/machine-learning-databases/00432/Data/News_Final.csv adatbázis tartalmazza (többek közt), hogy egy megjelent újságcikket, a rákövetkező héten hányan likeoltak a Facebookon. Hajts végre egy gépi tanulási kísérletet, hogy megtudjuk, hogy a `headline` szövege és a `source` alapján mennyire jól lehet megjósolni a `Facebook` likeok számát!

In [ ]:
# adat beolvasása
import pandas as pd
df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/00432/Data/News_Final.csv')
df


Csak azokat a cikkeket használd, ahol pozitív a `Facebook` likeok száma! Tanító adatbázisnak használd a 2016 máriusáig megjelent (`PublishDate`) cikkeket és értékeld ki a 2016. ápr 1. utáni cikkeken!

In [ ]:
# időpontként és ne string-ként kezeljük a PublishDate-et
df.PublishDate = pd.to_datetime(df.PublishDate)

# nem pozitív Facebook like értékkel bíró sorokat eldobjuk
df = df[df.Facebook>0]

In [ ]:
# Tanító és kiértékelő adatbázisra vágunk
traindf = df[df.PublishDate <= '2016-04-01']
testdf = df[df.PublishDate > '2016-04-01']
print("TanítóDB méretei:", traindf.shape)
print("KiértékelőDB méretei:", testdf.shape)

In [ ]:
# Jellemzőkinyerés a headline és source szöveges mezőkből
from sklearn.feature_extraction.text import CountVectorizer

bow_extractor = CountVectorizer()
train_features = bow_extractor.fit_transform(traindf.Headline + traindf.Source) # string oszlopok összeadása a sorok szövegeinek összefűzését jelenti
# ez elszáll a NaN-ok miatt...

In [ ]:
# dobjuk a NaNos sorokat
traindf = traindf.dropna()
testdf = testdf.dropna()
print("TanítóDB méretei:", traindf.shape)
print("KiértékelőDB méretei:", testdf.shape)
# nem sokat veszítettünk

In [ ]:
 # így már lefut
train_features = bow_extractor.fit_transform(traindf.Headline + traindf.Source)
print(train_features.shape)
# 55ezer szavunk=jellemzőnk van

In [ ]:
# Regressziós modell tanulása
from sklearn.linear_model import LinearRegression

reg = LinearRegression(normalize=True)
reg.fit(train_features, traindf.Facebook) # Facebook oszlop a célváltozó
# ez így nagyon lassú... inkább állítsuk le a futást.

In [ ]:
# Csökkentsük a tanító adatbázis méretét, hogy lássunk valami eredményt!
# vagy a sorok számát csökkentjük (traindf_small = traindf[:2000]) vagy a jellemzők számát
# nézzük a jellemzőszámot 55ezerről hogy lehet csökkenteni:
bow_extractor = CountVectorizer(min_df=10) # min_df=N esetén csak azok a szavak kerülnek be amelyek legalább N dokumentumban előfordulnak. A ritkák úgyis csak zajt hoznak be...
train_features = bow_extractor.fit_transform(traindf.Headline + traindf.Source)
print(train_features.shape) # így már csak 7254 jellemző maradt

In [ ]:
# és gyorsan lefut a tanulás a teljes halmazon is
reg = LinearRegression()
reg.fit(train_features, traindf.Facebook)

In [ ]:
# predikció és kiértékelés a kiértékelő adatbázisra
test_features = bow_extractor.transform(testdf.Headline + testdf.Source) # a transform a fit-nél kiválasztott szavakat használja csak
prediction = reg.predict(test_features)
from sklearn.metrics import mean_squared_error
mean_squared_error(prediction, testdf.Facebook)

In [ ]:
### Mindig hasonlítsuk az eredményt baselinehoz!
from sklearn.dummy import DummyRegressor
dummy = DummyRegressor(strategy='mean') # tanító adatbázis címkéinek átlaga lesz mindig a predikció
dummy.fit(train_features, traindf.Facebook)
mean_squared_error(dummy.predict(test_features), testdf.Facebook)

Sokkal rosszabb a tanuló, mint a baseline :(

Mivel szövegekről van szó, érdemes lehet kipróbálni a regressziós döntési fát is (talán ennyi jellemzőre még lefut).

In [ ]:
from sklearn.tree import DecisionTreeRegressor
reg = DecisionTreeRegressor(min_samples_leaf=5000) # döntési fa regresszióra

In [ ]:
reg.fit(train_features, traindf.Facebook)
prediction = reg.predict(test_features)
mean_squared_error(prediction, testdf.Facebook)
# veri a baselinet ...